In [ ]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/

In [ ]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (32,32)
Case_Num = 5  # Creating Number of classes


In [ ]:
# Function Definition --> For reading the Images and append it to a list\n
def Reading_Data():
    from tqdm import tqdm
    import cv2  # Used for Images processing.
    import shutil
    class_num = 0
    workdir = os.listdir(Work_Dir)
    if '.DS_Store' in workdir:
        workdir.remove('.DS_Store')
    for category in workdir:
        Category_Path = os.path.join(Work_Dir, category + '/')
        print(Category_Path)
        print(class_num)
        categorydir = os.listdir(Category_Path)
        print(categorydir)
        if '.DS_Store' in categorydir:
            categorydir.remove('.DS_Store')
        print(categorydir)
        for patient in categorydir:
            Patient_Path = os.path.join(Category_Path, patient + '/')
            # listing all images present in the category folder.
            for img in os.listdir(Patient_Path):
                try:
                    img_array = cv2.imread(os.path.join(
                        Patient_Path, img), cv2.IMREAD_GRAYSCALE)
                    # Saving images with their corresponding class labels.
                    data.append([img_array, class_num])
                except Exception as e:
                    print(e)
        class_num = 1
        shutil.rmtree(Category_Path)


In [ ]:
# Function Defination --> For Initilizing all features & labels of the processed image in the list X & Y

def Initilizing_Features_Labels():
    for features, label in data:
        X.append(features)
        y.append(label)

    print('List Size: ', len(X), len(y))


In [ ]:
# Function Defination --> Converting List into Numpy Array for faster calculation & it also take less space.

def Converting(block_h, block_w):
    import numpy as np  # Used for array operations.
    global X, y

    # -1 is added to solve dimension mismatch while converting list to an array.
    X = np.array(X).reshape((-1, block_h, block_w, 1))
    y = np.array(y)

    print('Array Size with Reshape: ', len(X), len(y))
    print('Array Shape with Reshape: ', X.shape, y.shape)


In [ ]:
# Function Definition --> For Storing Preprocessed data in the file.

def Storing_Preprocessed_data(X_train, X_test, y_train, y_test, case_number):
    # To save the data or Trained Deep learning model in the File.
    import pickle

    # Storing Features & Labels
    pickle_out = open(
        Work_Dir + 'Case_{}'.format(case_number) + '/X_train.pickle', 'wb')
    pickle.dump(X_train, pickle_out)  # Storing Training Features
    pickle_out.close()

    pickle_out = open(
        Work_Dir + 'Case_{}'.format(case_number) + '/X_test.pickle', 'wb')
    pickle.dump(X_test, pickle_out)  # Storing testing Features
    pickle_out.close()

    pickle_out = open(
        Work_Dir + 'Case_{}'.format(case_number) + '/y_train.pickle', 'wb')
    pickle.dump(y_train, pickle_out)  # Storing Training Labels
    pickle_out.close()

    pickle_out = open(
        Work_Dir + 'Case_{}'.format(case_number) + '/y_test.pickle', 'wb')
    pickle.dump(y_test, pickle_out)  # Storing testing Labels
    pickle_out.close()


In [ ]:
# Function Definition --> Dividing Datasets into number of cases for Cross Validation

def Creating_Cases(Case_Num):
    import shutil
    import os
    from sklearn.model_selection import KFold
    from sklearn.model_selection import RepeatedKFold

    for case in range(Case_Num):
        CASES.append('Case_' + str(case + 1))

    # Creating Cases
    try:
        for case in CASES:
            os.mkdir(Work_Dir + case)
    except Exception as e:
        pass

    kf = KFold(n_splits=5)
    case_number = 1
    for train_index, test_index in kf.split(X):

        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        Storing_Preprocessed_data(
            X_train, X_test, y_train, y_test, case_number)

        case_number = case_number + 1


In [ ]:
# All Function Calls
import os
import random
import numpy as np  # Used for array operations.
import pickle

PATH = path + 'Data/UPENN/'
Work_Dir = PATH + 'Working_Data/'

# Calling Functions:

# Creating list for storing processed data
data = []

# Function Call --> For reading the Images and append it to a list
Reading_Data()

# Printing Length of the total data.
print('Length of the total data: ' + str(len(data)))

# Randomly Shuffling and printing data for unbiased model.
random.shuffle(data)
for sample in data[:50]:
    print(sample[1], end=' ')

# Initilizing all features & labels of the processed image in the list X & Y.
X = []
y = []

# Function Call --> For Initilizing all features & labels of the processed image in the list X & Y
Initilizing_Features_Labels()

# Function Call --> For Converting List into Numpy Array for faster calculation & it also take less space.
Converting(block_h, block_w)

# Function Call --> Dividing Datasets into number of cases for Cross Validation
CASES = []
Creating_Cases(Case_Num)

os.chdir(PATH)
print('Done..!!')
